In [12]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [13]:
test_day = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D10.parquet')
test_day.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.279469,985,ABEV3.SA,NaN,0.520882,-1.0,0.520882,-1.0,0.520882,0.511152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.265302,227,ABEV3.SA,-0.026404,0.519562,-1.0,0.494478,1.0,0.501104,0.510650,0.026404,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000030,-0.000016,NaN,-227
2,2000-01-12,0.481293,0.481293,0.481293,0.481293,0.258228,1137,ABEV3.SA,-0.013185,0.517648,-1.0,0.481293,1.0,0.514289,0.510832,0.013185,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000097,-0.000049,NaN,-1364
3,2000-01-13,0.484589,0.484589,0.484589,0.484589,0.259996,606,ABEV3.SA,0.003296,0.515995,-1.0,0.484589,1.0,0.485255,0.509553,0.000000,0.003296,9.538366,NaN,NaN,NaN,NaN,-0.000185,-0.000095,NaN,-1970
4,2000-01-14,0.494478,0.494478,0.494478,0.494478,0.265302,6445,ABEV3.SA,0.009889,0.514920,-1.0,0.494478,-1.0,0.494478,0.508799,0.000000,0.009889,31.994220,NaN,NaN,NaN,NaN,-0.000272,-0.000148,NaN,-8415


In [14]:
test_day['Ticker'].unique()

array(['ABEV3.SA', 'ITUB4.SA', 'MGLU3.SA', 'PETR4.SA', 'VALE3.SA'],
      dtype=object)

In [15]:
test_day_AMBEV = test_day[test_day['Ticker'] == 'ABEV3.SA'].reset_index(drop=True)
test_day_AMBEV['Prediction'] = np.NaN

test_day_AMBEV.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.279469,985,ABEV3.SA,NaN,0.520882,NaN,0.520882,-1.0,0.520882,0.511152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.265302,227,ABEV3.SA,-0.026404,0.519562,NaN,0.494478,1.0,0.501104,0.510650,0.026404,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000030,-0.000016,NaN,-227
2,2000-01-12,0.481293,0.481293,0.481293,0.481293,0.258228,1137,ABEV3.SA,-0.013185,0.517648,NaN,0.481293,1.0,0.514289,0.510832,0.013185,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000097,-0.000049,NaN,-1364
3,2000-01-13,0.484589,0.484589,0.484589,0.484589,0.259996,606,ABEV3.SA,0.003296,0.515995,NaN,0.484589,1.0,0.485255,0.509553,0.000000,0.003296,9.538366,NaN,NaN,NaN,NaN,-0.000185,-0.000095,NaN,-1970
4,2000-01-14,0.494478,0.494478,0.494478,0.494478,0.265302,6445,ABEV3.SA,0.009889,0.514920,NaN,0.494478,-1.0,0.494478,0.508799,0.000000,0.009889,31.994220,NaN,NaN,NaN,NaN,-0.000272,-0.000148,NaN,-8415


In [16]:
test_day_AMBEV['Date'] = pd.to_datetime(test_day_AMBEV['Date'])

# Definir a data inicial para filtrar
inicio = '2024-12-05'  # Data inicial

# Filtrar as linhas onde a data é maior ou igual a 'inicio'
test_day_AMBEV = test_day_AMBEV[test_day_AMBEV['Date'] >= inicio]

test_day_AMBEV.head(15)


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5717,2024-12-05,14.05,14.46,14.00,14.42,13.666730,38794000,ABEV3.SA,0.490000,12.899844,NaN,14.42,-1.0,12.78,13.074744,0.000000,0.49,86.722945,12.25,14.46,29.404700,-70.595300,0.003607,-0.009430,0.015191,-2545143975
5718,2024-12-06,14.30,14.53,14.15,14.22,13.477178,46449200,ABEV3.SA,-0.200000,12.965852,NaN,14.22,-1.0,12.23,13.032506,0.200000,0.00,77.627058,12.25,14.53,31.397008,-68.602992,0.019567,-0.003630,0.021056,-2498694775
5719,2024-12-09,14.22,14.33,14.14,14.21,13.467700,25321800,ABEV3.SA,-0.010000,13.028059,NaN,14.21,-1.0,12.22,12.991881,0.010000,0.00,77.160149,12.25,14.53,34.125404,-65.874596,0.036811,0.004458,0.026312,-2473372975
5720,2024-12-10,14.33,14.50,14.27,14.27,13.524566,22942500,ABEV3.SA,0.060000,13.090156,NaN,14.27,-1.0,12.14,12.949287,0.000000,0.06,78.073184,12.25,14.53,36.848960,-63.151040,0.054855,0.014537,0.031789,-2450430475
5721,2024-12-11,14.27,14.36,13.66,13.80,13.079118,49150600,ABEV3.SA,-0.470000,13.125648,NaN,13.80,-1.0,12.07,12.905323,0.470000,0.00,57.351316,12.25,14.53,38.405635,-61.594365,0.071199,0.025870,0.035758,-2401279875
5722,2024-12-12,13.50,13.58,13.10,13.10,12.415685,49616900,ABEV3.SA,-0.700000,13.124366,NaN,13.10,-1.0,11.88,12.854057,0.700000,0.00,39.386518,12.25,14.53,38.349390,-61.650610,0.083090,0.037314,0.035422,-2450896775
5723,2024-12-13,13.10,13.36,13.01,13.08,12.396729,30867900,ABEV3.SA,-0.020000,13.122148,NaN,13.08,NaN,NaN,NaN,0.020000,0.00,38.983936,12.25,14.53,38.252096,-61.747904,0.091282,0.048108,0.032906,-2481764675
5724,2024-12-16,13.08,13.42,12.84,12.84,12.169266,32095500,ABEV3.SA,-0.240000,13.108040,NaN,12.84,NaN,NaN,NaN,0.240000,0.00,34.150823,12.25,14.53,37.633351,-62.366649,0.095535,0.057593,0.027163,-2513860175
5725,2024-12-17,13.00,13.19,12.99,13.10,12.415685,39564500,ABEV3.SA,0.260000,13.107638,NaN,13.10,NaN,NaN,NaN,0.000000,0.26,42.986323,12.25,14.53,37.615720,-62.384280,0.097747,0.065624,0.022450,-2553424675
5726,2024-12-18,13.07,13.08,12.75,12.76,12.093445,40864200,ABEV3.SA,-0.340000,13.090256,NaN,12.76,NaN,NaN,NaN,0.340000,0.00,35.748723,12.25,14.53,36.853355,-63.146645,0.096979,0.071895,0.014761,-2594288875


##### Carregar modelo aprendido geral

In [17]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model.pkl')

#### Carregar modelo aprendido especifico.

In [17]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model_Ambev.pkl')

In [18]:
Features = test_day_AMBEV[['RSI','k_percent','r_percent','Price_Rate_Of_Change','MACD','MACD_EMA','On Balance Volume']]
Features.head()

,RSI,k_percent,r_percent,Price_Rate_Of_Change,MACD,MACD_EMA,On Balance Volume
5717,86.722945,29.404700,-70.595300,0.015191,0.003607,-0.009430,-2545143975
5718,77.627058,31.397008,-68.602992,0.021056,0.019567,-0.003630,-2498694775
5719,77.160149,34.125404,-65.874596,0.026312,0.036811,0.004458,-2473372975
5720,78.073184,36.848960,-63.151040,0.031789,0.054855,0.014537,-2450430475
5721,57.351316,38.405635,-61.594365,0.035758,0.071199,0.025870,-2401279875


In [19]:
Features_array = Features.to_numpy()

# Fazer previsões usando o modelo carregado
predictions = rand_frst_clf_Ambev.predict(Features_array)

# Adicionar as previsões em uma nova coluna chamada 'Prediction'
test_day_AMBEV['Prediction'] = predictions

test_day_AMBEV.tail(30)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5717,2024-12-05,14.05,14.46,14.00,14.42,13.666730,38794000,ABEV3.SA,0.490000,12.899844,1.0,14.42,-1.0,12.78,13.074744,0.000000,0.49,86.722945,12.25,14.46,29.404700,-70.595300,0.003607,-0.009430,0.015191,-2545143975
5718,2024-12-06,14.30,14.53,14.15,14.22,13.477178,46449200,ABEV3.SA,-0.200000,12.965852,1.0,14.22,-1.0,12.23,13.032506,0.200000,0.00,77.627058,12.25,14.53,31.397008,-68.602992,0.019567,-0.003630,0.021056,-2498694775
5719,2024-12-09,14.22,14.33,14.14,14.21,13.467700,25321800,ABEV3.SA,-0.010000,13.028059,-1.0,14.21,-1.0,12.22,12.991881,0.010000,0.00,77.160149,12.25,14.53,34.125404,-65.874596,0.036811,0.004458,0.026312,-2473372975
5720,2024-12-10,14.33,14.50,14.27,14.27,13.524566,22942500,ABEV3.SA,0.060000,13.090156,-1.0,14.27,-1.0,12.14,12.949287,0.000000,0.06,78.073184,12.25,14.53,36.848960,-63.151040,0.054855,0.014537,0.031789,-2450430475
5721,2024-12-11,14.27,14.36,13.66,13.80,13.079118,49150600,ABEV3.SA,-0.470000,13.125648,-1.0,13.80,-1.0,12.07,12.905323,0.470000,0.00,57.351316,12.25,14.53,38.405635,-61.594365,0.071199,0.025870,0.035758,-2401279875
5722,2024-12-12,13.50,13.58,13.10,13.10,12.415685,49616900,ABEV3.SA,-0.700000,13.124366,-1.0,13.10,-1.0,11.88,12.854057,0.700000,0.00,39.386518,12.25,14.53,38.349390,-61.650610,0.083090,0.037314,0.035422,-2450896775
5723,2024-12-13,13.10,13.36,13.01,13.08,12.396729,30867900,ABEV3.SA,-0.020000,13.122148,-1.0,13.08,NaN,NaN,NaN,0.020000,0.00,38.983936,12.25,14.53,38.252096,-61.747904,0.091282,0.048108,0.032906,-2481764675
5724,2024-12-16,13.08,13.42,12.84,12.84,12.169266,32095500,ABEV3.SA,-0.240000,13.108040,-1.0,12.84,NaN,NaN,NaN,0.240000,0.00,34.150823,12.25,14.53,37.633351,-62.366649,0.095535,0.057593,0.027163,-2513860175
5725,2024-12-17,13.00,13.19,12.99,13.10,12.415685,39564500,ABEV3.SA,0.260000,13.107638,-1.0,13.10,NaN,NaN,NaN,0.000000,0.26,42.986323,12.25,14.53,37.615720,-62.384280,0.097747,0.065624,0.022450,-2553424675
5726,2024-12-18,13.07,13.08,12.75,12.76,12.093445,40864200,ABEV3.SA,-0.340000,13.090256,-1.0,12.76,NaN,NaN,NaN,0.340000,0.00,35.748723,12.25,14.53,36.853355,-63.146645,0.096979,0.071895,0.014761,-2594288875


In [9]:
test_day_AMBEV.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)

In [20]:
# We need to remove all rows that have an NaN value.
print('Antes de tirar as NaN nós tinhamos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))

# Any row that has a `NaN` value will be dropped.
test_day_AMBEV = test_day_AMBEV.dropna()

# Display how much we have left now.
print('Depois da retirada dos NaN temos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))


Antes de tirar as NaN nós tinhamos 16 linhas e 26 colunas
Depois da retirada dos NaN temos 6 linhas e 26 colunas


In [21]:
# Comparar valores e calcular a precisão
test_day_AMBEV['Match'] = (test_day_AMBEV['Prediction'] == test_day_AMBEV['Verify']).astype(int)
accuracy = test_day_AMBEV['Match'].mean()  # Proporção de acertos (iguais / total)

print(f"\nPrecisão do modelo: {accuracy:.2%}")


Precisão do modelo: 66.67%
